### 目標
Jambo 資料前處理

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import random

import re
from collections import Counter

### jambo的資料集介紹
* 本組使用的檔案為 jambo_6.csv
* 檔案資料大小為 18753 rows × 3 columns
* 3 個 columns 分別為 "channel_name"、"商品id"、"商品名稱"

**先利用蝦皮的資料建立好模型後，再將再將jambo的資料當作測試及進行預測**

In [ ]:
# import jambo data
df = pd.read_csv('/content/gdrive/Shareddrives/文字探勘/期中考/jambo_6.csv')
df

,channel_name,商品id,商品名稱
0,邦成-自倉（C倉出貨付款全開）,3102326,大蘋 0420 A01-(限+3)(10盒)韓國低分子海洋膠原蛋白粉 15入/盒*10
1,邦成-自倉（C倉出貨付款全開）,2764951,新三郎 0314 A47-強力磁鐵掛勾(裸裝)*1
2,邦成-自倉（C倉出貨付款全開）,3135009,舊三郎 0424 A26-(2條)AUS LIFE澳思萊蝸牛膠原彈潤身體修護霜100ml*2
3,邦成-自倉（C倉出貨付款全開）,3981214,舊三郎 0718 G02-速樂活 衣物香氛豆膠囊(隨機)*1
4,邦成-自倉（C倉出貨付款全開）,2951584,新三郎 0403 A14-(6件組)公主派對。極速孅 印加果油 (30顆/盒)*4+公主派對...
...,...,...,...
18747,邦成-自倉（C倉出貨付款全開）,2967713,（單獨出貨）888 0406 A06-(單獨出貨)(12件組)澎沛黑木耳900ML(冬瓜*2...
18748,蔥媽媽直播,5313116,第 013 標 (11/17)蔥水餃
18749,TR Box寶藏屋：傘的專家、居的職人,3944880,第 188 標(7/14)迪士尼冰壩杯(款式隨機)870ML一支168元
18750,邦成-自倉（C倉出貨付款全開）,5614550,888 1213 A02-蘭蔻 超未來肌因賦活露100ml(效期2023/11)*1


### 資料前處理

In [ ]:
product_name = df['商品名稱'].values
product_name

array(['大蘋 0420 A01-(限+3)(10盒)韓國低分子海洋膠原蛋白粉 15入/盒*10',
       '新三郎 0314 A47-強力磁鐵掛勾(裸裝)*1',
       '舊三郎 0424 A26-(2條)AUS LIFE澳思萊蝸牛膠原彈潤身體修護霜100ml*2', ...,
       '第 188 標(7/14)迪士尼冰壩杯(款式隨機)870ML一支168元',
       '888 1213 A02-蘭蔻 超未來肌因賦活露100ml(效期2023/11)*1',
       '畢哥 0920 A28-(2個)YC-7314 加壓護膝(單入)*2'], dtype=object)

#### 去除括號內容

In [ ]:
for i in range(len(product_name)):

  # 去除【】,［］, [], （）, (), 《》, 〔〕, 「」內容
  removed = re.sub(u"\【.*?\】", '', product_name[i])
  removed = re.sub(u"\［.*?\］", '', removed)
  removed = re.sub(u"\[.*?\]", '', removed)
  removed = re.sub(u"\（.*?\）", '', removed)
  removed = re.sub(u"\(.*?\)", '', removed)
  removed = re.sub(u"\《.*?\》", '', removed)
  removed = re.sub(u"\〔.*?\〕", '', removed)
  removed = re.sub(u"\「.*?\」", '', removed)

  product_name[i] = removed

#### 停用詞移除
Stop words：
1. 空格
2. ｜, |, │, ︱, （, (, ), 【, /, ／,&,＆,.,+,＋,#,＃,$,＄（其中包含每個符號的全型以及半型）<br>
3. 與產品無相關的詞：入、組、現貨、折

策略：將 stop words 移除

In [ ]:
stop_word = [' ','｜','|','│','︱','（','(',')','【','/','／','&','＆','.','+','＋','#','＃','$','＄','--','-','支','入','組','元','一盒','台','一張','一條','一捲','一個','大張','隻','現貨','折','第標']
for i in range(len(product_name)):

  for j in stop_word:
    if j in product_name[i]:
      removed_stop_word = product_name[i].replace(j, '')
      product_name[i] = removed_stop_word

#### 處理非中文的資料
策略：
1. 移除大部分英文以及數字
2. 判別有 T 的是否為 吸排t、t恤、素t、短袖上衣、寬鬆短袖、大學 T

In [ ]:
# 判斷字串是否為中文
def is_all_chinese(strs):
  for _char in strs:
    if not '\u4e00' <=_char <= '\u9fa5':
      return False
  return True

In [ ]:
pl = []

# 將t恤的t字與tee改為「踢」
for product in product_name:
  product = product.lower()
  
  for i, char in enumerate(product):
    # t前後是中文 eg."素t短袖上衣寬鬆短袖"
    if char == 't' and 0 < i < (len(product) - 1):
        if (is_all_chinese(product[i-1]) and is_all_chinese(product[i+1])):
            p = list(product)
            p[i] = '踢'
            product = ''.join(p)
    # t在最前面 t在最後面 eg."t恤大學t"
    elif char == 't' and ((i == 0 and is_all_chinese(product[i+1])) or (i == (len(product) - 1) and is_all_chinese(product[i-1]))):
        p = list(product)
        p[i] = '踢'
        product = ''.join(p)
    # "tee"
    elif char == 't' and 0 <= i <= (len(product) - 3) and product[i+1] == 'e' and product[i+2] == 'e':
      p = list(product)
      p[i] = '踢'
      product = ''.join(p)

    if is_all_chinese(product[i]) == False:
        p = list(product)
        p[i] = ' '
        product = ''.join(p)
  pl.append(product)

In [ ]:
# 移除剩餘的空格
stop_word = [' ']
for i in range(len(pl)):

  for j in stop_word:
    if j in pl[i]:
      removed_stop_word = pl[i].replace(j, '')
      pl[i] = removed_stop_word

In [ ]:
# 處理沒有值的資料
for i in range(len(pl)):
    if(pl[i]==''):
        pl[i]="X"

In [ ]:
df['product_name'] = pl
df

,channel_name,商品id,商品名稱,product_name
0,邦成-自倉（C倉出貨付款全開）,3102326,大蘋0420A01韓國低分子海洋膠原蛋白粉15盒*10,大蘋韓國低分子海洋膠原蛋白粉盒
1,邦成-自倉（C倉出貨付款全開）,2764951,新三郎0314A47強力磁鐵掛勾*1,新三郎強力磁鐵掛勾
2,邦成-自倉（C倉出貨付款全開）,3135009,舊三郎0424A26AUSLIFE澳思萊蝸牛膠原彈潤身體修護霜100ml*2,舊三郎澳思萊蝸牛膠原彈潤身體修護霜
3,邦成-自倉（C倉出貨付款全開）,3981214,舊三郎0718G02速樂活衣物香氛豆膠囊*1,舊三郎速樂活衣物香氛豆膠囊
4,邦成-自倉（C倉出貨付款全開）,2951584,新三郎0403A14公主派對。極速孅印加果油*4公主派對防彈奶茶*1iSense智能電子體重秤*1,新三郎公主派對極速孅印加果油公主派對防彈奶茶智能電子體重秤
...,...,...,...,...
18747,邦成-自倉（C倉出貨付款全開）,2967713,8880406A06澎沛黑木耳900ML澎沛白木耳900ML,澎沛黑木耳澎沛白木耳
18748,蔥媽媽直播,5313116,第013標蔥水餃,第標蔥水餃
18749,TR Box寶藏屋：傘的專家、居的職人,3944880,第188標迪士尼冰壩杯870ML一168,第標迪士尼冰壩杯一
18750,邦成-自倉（C倉出貨付款全開）,5614550,8881213A02蘭蔻超未來肌因賦活露100ml*1,蘭蔻超未來肌因賦活露


In [ ]:
jambo_product_name = df['product_name'].values

In [ ]:
!pip install ckiptagger

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from ckiptagger import WS

In [ ]:
# 導入CKIPtagger 斷詞模型
ws = WS(r'/content/gdrive/Shareddrives/文字探勘/Lab3/data')

/usr/local/lib/python3.7/dist-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)


In [ ]:
# 執行斷詞
# 時間：15 mins
ckipResults = []
for m in jambo_product_name:
  ws_results = ws([m])
  ckipResults.append(ws_results)
  # print(ws_results)

In [ ]:
# 將斷詞結果利用空格分開
ckip_strings = []
for i in range(len(ckipResults)):
    ckip_string = ''
    for j in range(len(ckipResults[i])):
      for v in range(len(ckipResults[i][j])):
          ckip_string = ckip_string + ckipResults[i][j][v]
          if v < (len(ckipResults[i][j]) - 1):
              ckip_string = ckip_string + ' '
      ckip_strings.append(ckip_string)

In [ ]:
df['ckip_results'] = ckipResults
df['ckip_string'] = ckip_strings
df

,channel_name,商品id,商品名稱,product_name,ckip_results,ckip_string
0,邦成-自倉（C倉出貨付款全開）,3102326,大蘋0420A01韓國低分子海洋膠原蛋白粉15盒*10,大蘋韓國低分子海洋膠原蛋白粉盒,"[[大蘋, 韓國, 低分子, 海洋, 膠原, 蛋白, 粉盒]]",大蘋 韓國 低分子 海洋 膠原 蛋白 粉盒
1,邦成-自倉（C倉出貨付款全開）,2764951,新三郎0314A47強力磁鐵掛勾*1,新三郎強力磁鐵掛勾,"[[新三郎, 強力, 磁鐵, 掛勾]]",新三郎 強力 磁鐵 掛勾
2,邦成-自倉（C倉出貨付款全開）,3135009,舊三郎0424A26AUSLIFE澳思萊蝸牛膠原彈潤身體修護霜100ml*2,舊三郎澳思萊蝸牛膠原彈潤身體修護霜,"[[舊三郎, 澳思萊, 蝸牛, 膠, 原, 彈潤, 身體, 修護, 霜]]",舊三郎 澳思萊 蝸牛 膠 原 彈潤 身體 修護 霜
3,邦成-自倉（C倉出貨付款全開）,3981214,舊三郎0718G02速樂活衣物香氛豆膠囊*1,舊三郎速樂活衣物香氛豆膠囊,"[[舊三郎, 速樂活, 衣物, 香氛, 豆膠囊]]",舊三郎 速樂活 衣物 香氛 豆膠囊
4,邦成-自倉（C倉出貨付款全開）,2951584,新三郎0403A14公主派對。極速孅印加果油*4公主派對防彈奶茶*1iSense智能電子體重秤*1,新三郎公主派對極速孅印加果油公主派對防彈奶茶智能電子體重秤,"[[新三郎, 公主派, 對, 極速, 孅, 印加, 果油, 公主派, 對, 防彈, 奶茶, ...",新三郎 公主派 對 極速 孅 印加 果油 公主派 對 防彈 奶茶 智能 電子 體重 秤
...,...,...,...,...,...,...
18747,邦成-自倉（C倉出貨付款全開）,2967713,8880406A06澎沛黑木耳900ML澎沛白木耳900ML,澎沛黑木耳澎沛白木耳,"[[澎沛, 黑木耳, 澎沛, 白木耳]]",澎沛 黑木耳 澎沛 白木耳
18748,蔥媽媽直播,5313116,第013標蔥水餃,第標蔥水餃,"[[第, 標蔥, 水餃]]",第 標蔥 水餃
18749,TR Box寶藏屋：傘的專家、居的職人,3944880,第188標迪士尼冰壩杯870ML一168,第標迪士尼冰壩杯一,"[[第, 標, 迪士尼, 冰壩杯, 一]]",第 標 迪士尼 冰壩杯 一
18750,邦成-自倉（C倉出貨付款全開）,5614550,8881213A02蘭蔻超未來肌因賦活露100ml*1,蘭蔻超未來肌因賦活露,"[[蘭蔻, 超, 未來, 肌因, 賦活, 露]]",蘭蔻 超 未來 肌因 賦活 露


In [ ]:
# 匯出資料
df.to_csv('jambo_6_preprocessing_ckip.csv', encoding='utf_8_sig')